# TRANSACTIONS PROCESSOR

In Rewards-R-Tokens, the transactions processor is primarily responsible for:

1. Mocking credit card transactions
2. Calculate appropriate rewards token count
3. Interact with deployed smart contract
4. Transfer token to the customer address.

A credit card transaction in our blockchain world carries the following schema:

```code=json
    {
    transaction_id: int,
    customer_id: address_from_ganache,
    transaction_amount: int(in dollars),
    transaction_category: ['gas', 'restaurant', 'grocery', 'subscription', 'entertainment'],
    transaction_date: Date
    }
```

In [1]:
# Project Imports

from time import sleep
from web3 import Web3, Account
from dotenv import load_dotenv
from random import choice
from datetime import datetime

import os
import json
import random

In [2]:
# Declare local Ganache HTTP provider
local_ganache_http_provider = 'HTTP://127.0.0.1:7545'

# Connect to the local Ganache HTTP provider
w3=Web3(Web3.HTTPProvider(local_ganache_http_provider))

In [3]:
"""
!!! ATTENTION !!! ATTENTION !!! ATTENTION !!!

.env file must be present in the current working directory
wuth the following key:value pairs:

CONTRACT_ADDRESS=<Deployed Contact Address from Remix>
CONTRACT_OWNER_ADDRESS=<Contract Owner Address from Ganache>
CONTRACT_OWNER_PVT_KEY=<Private key of CONTRACT_OWNER_ADDRESS from Ganache>

You should pick the address of account that was used to deploy the contract 
as CONTRACT_OWNER_ADDRESS from Ganache.
"""


load_dotenv()
contract_address = os.getenv('CONTRACT_ADDRESS')
contract_owner_address = os.getenv('CONTRACT_OWNER_ADDRESS')
contract_owner_pvt_key = os.getenv('CONTRACT_OWNER_PVT_KEY')
print(f'Contract Deployed at: {contract_address} by {contract_owner_address}')

Contract Deployed at: 0x1e118cC53da662dDc0C28CB5119833eF4168fC95 by 0x03Cc221e329d0CAcd5A57C6E117b77220b64C070


In [4]:
# Load Application Binary Interface (abi) file as JSON

with open('contract_abi.txt') as contract_abi_file:
    contract_abi_json = json.load(contract_abi_file)

# Connect to the deployed contract
rt_contract = w3.eth.contract(contract_address, abi=contract_abi_json)

In [5]:
# Get all accounts from local Ganache provider
accounts = w3.eth.accounts

# Remove contract owner address from accounts as we
# DO NOT want to use the owner address for transacations.
accounts.remove(contract_owner_address)

In [6]:
# Create a dictionary for total customer spendings
customer_totals = {}

In [7]:
# Declare of list of transaction categories
txn_categories = ['gas', 'restaurant',  'grocery', 'subscription', 'entertainment']

In [8]:
# Declare custom token conversion values for categories.
# TODO: This is a static list but this would be modified to be served via an API
#  to make it dynamic in future.

txn_categories_conversion_rate = { 
                                  'gas': 2.0,
                                  'restaurant': 1.5, 
                                  'grocery': 1.0,
                                  'subscription': 3.0,
                                  'entertainment': 2.5
                                 }

# Number of tokens awarded = transacation_amount * txn_categories_conversion_rate[transacation_category]
# This basically means the following:
# If a customer spends $10 on gas, they would get 20 tokens
# =====> $10 (transacation_amount) * 2.0 (txn_categories_conversion_rate['gas']))
# If a customer spends $9.99 on subscriptions, they would get 29.97 tokens 
# =====> $9.99 (transacation_amount) * 3.0 (txn_categories_conversion_rate['subscription']))

In [9]:
# Define a generator function to mock credit card transactions:

def credit_card_transactions(num_transacations=10):
    for i in range(num_transacations):
        tx = {}
        tx['id'] = i
        tx['customer_id'] = choice(accounts)
        tx['amount'] = round(random.uniform(0.5, 99.5), 2)
        tx['category'] = choice(txn_categories)
        tx['date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        yield tx
    

In [10]:
for c_txn in credit_card_transactions(50):
    num_tokens_award = c_txn['amount'] * txn_categories_conversion_rate[c_txn['category']]
    num_tokens_award = int(num_tokens_award) * (10**18)
    txn = {
    'from': contract_owner_address,
    'nonce': w3.eth.get_transaction_count(contract_owner_address),
    'gasPrice': w3.eth.gas_price
    }
    
    raw_txn = rt_contract.functions.mint(c_txn['customer_id'], num_tokens_award).buildTransaction(txn)
    
    signed_txn = w3.eth.account.signTransaction(raw_txn, private_key=contract_owner_pvt_key)
    
    txn_response = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    
    print(f"{num_tokens_award/(10**18)} tokens awarded to {c_txn['customer_id']} for spending ${c_txn['amount']} on {c_txn['category']}.")
    
    sleep(1.0)

244.0 tokens awarded to 0xB7f399244620EcDd20A8b231DF65FaBb11FC2A65 for spending $97.78 on entertainment.
21.0 tokens awarded to 0xf93cEDcFcFde749ec876698185fA090E6880CA07 for spending $10.68 on gas.
136.0 tokens awarded to 0x6781C4D10aE1A586baAB792ca85200eF204c6779 for spending $54.62 on entertainment.
105.0 tokens awarded to 0xCFe793AcEea7525764403b99cbDd60c36892242C for spending $42.15 on entertainment.
27.0 tokens awarded to 0xF09bBa9C791f6898D45Ae761d410D774BE5D52F0 for spending $27.07 on grocery.
123.0 tokens awarded to 0xaE8c4Ef75f9fb0Ffae5578F912B1BbE3b8ce493e for spending $41.12 on subscription.
35.0 tokens awarded to 0x7DA90F3794c71817f3744994DaF762f5b1E4079c for spending $14.06 on entertainment.
13.0 tokens awarded to 0xA81dd6b3dcdE173C76B43de2CC60998cd1779Cda for spending $13.33 on grocery.
135.0 tokens awarded to 0xCFe793AcEea7525764403b99cbDd60c36892242C for spending $45.21 on subscription.
72.0 tokens awarded to 0x6781C4D10aE1A586baAB792ca85200eF204c6779 for spending $28.

In [11]:
def get_account_balance(account_id):
    return rt_contract.functions.balanceOf(account_id).call()

In [12]:
print('Account Balance')
print('===============')

for account_no in accounts:
    print(f"{account_no}: {get_account_balance(account_no)}")

Account Balance
0xF09bBa9C791f6898D45Ae761d410D774BE5D52F0: 317000000000000000000
0xB7f399244620EcDd20A8b231DF65FaBb11FC2A65: 1834000000000000000000
0xD69c0387909fA62a6C621e3A564Bf6e5F518CFc5: 1507000000000000000000
0xA81dd6b3dcdE173C76B43de2CC60998cd1779Cda: 1133000000000000000000
0xCFe793AcEea7525764403b99cbDd60c36892242C: 1546000000000000000000
0x6781C4D10aE1A586baAB792ca85200eF204c6779: 824000000000000000000
0xaE8c4Ef75f9fb0Ffae5578F912B1BbE3b8ce493e: 1103000000000000000000
0xf93cEDcFcFde749ec876698185fA090E6880CA07: 1412000000000000000000
0x7DA90F3794c71817f3744994DaF762f5b1E4079c: 1143000000000000000000


In [13]:
dir(rt_contract.functions.transferFrom)

['__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_encode_transaction_data',
 '_return_data_normalizers',
 '_set_function_info',
 'abi',
 'address',
 'args',
 'arguments',
 'buildTransaction',
 'call',
 'contract_abi',
 'estimateGas',
 'factory',
 'fn_name',
 'function_identifier',
 'kwargs',
 'transact',
 'transaction',
 'web3']